In [49]:
import os
import uuid

def get_user_id():
    user_id_file = "user_ids.txt"  # File để lưu nhiều user_id
    
    # Tạo user_id mới
    new_user_id = str(uuid.uuid4())[:8]  # Lấy 8 ký tự đầu của UUID
    
    # Lưu user_id mới vào file
    with open(user_id_file, "a") as f:  # Mở file ở chế độ append (thêm vào cuối)
        f.write(new_user_id + "\n")  # Ghi user_id mới và xuống dòng
    
    return new_user_id

# Lấy user_id mới
user_id = get_user_id()
print(f"🔑 User ID mới của bạn: {user_id}")

🔑 User ID mới của bạn: c3706428


In [1]:
import os
import uuid
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd

job_skills = pd.read_csv('C:/HocTap/DoAnTotNghiep/archive/job_skills.csv')
job_summary = pd.read_csv('C:/HocTap/DoAnTotNghiep/archive/job_summary.csv')
job_posts = pd.read_csv('C:/HocTap/DoAnTotNghiep/archive/linkedin_job_postings.csv')

In [ ]:
df = job_posts.merge(job_summary, on="job_link", how="left").merge(job_skills, on="job_link", how="left")

In [ ]:
df.dropna(subset=['company', 'job_location'], inplace=True)

In [5]:
# Tạo job profile bằng cách kết hợp các cột quan trọng
df["job_profile"] = df["job_title"] + " " + df["job_summary"] + " " + df["job_skills"] + " " + df["job_level"] + " " + df["job_type"]

In [6]:
print(df["job_profile"].head())

0    Account Executive - Dispensing (NorCal/Norther...
1    Registered Nurse - RN Care Manager Employment ...
2    RESTAURANT SUPERVISOR - THE FORKLIFT Job Detai...
3    Independent Real Estate Agent Who We Are\nRand...
4                                                  NaN
Name: job_profile, dtype: object


In [7]:
print(df["job_profile"].isnull().sum())

54156


In [8]:
print("Tổng số giá trị trong cột job_profile:", df["job_profile"].count())

Tổng số giá trị trong cột job_profile: 1294268


In [9]:
print("Số lượng giá trị NaN trong từng cột:")
print(df[["job_title", "job_summary", "job_skills", "job_level", "job_type"]].isnull().sum())

Số lượng giá trị NaN trong từng cột:
job_title          0
job_summary    51120
job_skills     54156
job_level          0
job_type           0
dtype: int64


In [10]:
df.dropna(subset=["job_profile"], inplace=True)

In [14]:
# Lấy danh sách kỹ năng, cấp bậc, vị trí hợp lệ
valid_skills = set(df["job_skills"].str.split(",").explode().str.strip().dropna().unique())
valid_levels = set(df["job_level"].dropna().unique())
valid_locations = set(df["job_location"].dropna().unique())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")
job_vectors = vectorizer.fit_transform(df["job_profile"])

print("TF-IDF vectorization hoàn tất.")

TF-IDF vectorization hoàn tất.


In [21]:
from sklearn.metrics.pairwise import cosine_similarity

candidate_skills = input("Nhập kỹ năng của bạn: ")
candidate_level = input("Nhập cấp bậc mong muốn: ")
candidate_location = input("Nhập vị trí mong muốn: ")

invalid_inputs = []
input_skills = set(map(str.strip, candidate_skills.split(",")))
if not input_skills.issubset(valid_skills):
    invalid_inputs.append("Kỹ năng không tồn tại trong hệ thống!")

if candidate_level not in valid_levels:
    invalid_inputs.append("Cấp bậc không hợp lệ!")

if candidate_location not in valid_locations:
    invalid_inputs.append("Vị trí không hợp lệ!")

if invalid_inputs:
    print("❌ Lỗi nhập liệu:")
    for error in invalid_inputs:
        print(f" - {error}")
else:
    candidate_profile = f"I am looking for a {candidate_level} job in {candidate_location}. My key skills are {candidate_skills}."

    # Chuyển đổi thông tin ứng viên thành vector
    candidate_vector = vectorizer.transform([candidate_profile])

    # Tính cosine similarity giữa ứng viên và các công việc
    similarity_scores = cosine_similarity(candidate_vector, job_vectors).flatten()

    top_n = 5
    top_jobs_indices = similarity_scores.argsort()[::-1][:top_n]

    # Lấy danh sách công việc phù hợp
    top_jobs = df.iloc[top_jobs_indices][["job_link", "job_title", "company", "job_location"]]

    print("Công việc được gợi ý:")
    print(top_jobs)


Công việc được gợi ý:
                                                  job_link  \
740787   https://www.linkedin.com/jobs/view/nurse-pract...   
244230   https://www.linkedin.com/jobs/view/business-an...   
804488   https://www.linkedin.com/jobs/view/us-operatio...   
1147830  https://www.linkedin.com/jobs/view/veterinaria...   
576801   https://www.linkedin.com/jobs/view/data-qualit...   

                                                 job_title  \
740787   Nurse Practitioner PRN- positions available st...   
244230                                Business Analyst III   
804488   US Operational Health, Safety, Security & Envi...   
1147830  Veterinarian - Mentorship, Training, and Auton...   
576801                       Data Quality Business Analyst   

                                        company  \
740787   Northeast Healthcare Recruitment, Inc.   
244230               Oloop Technology Solutions   
804488                           Energy Jobline   
1147830                     

In [53]:
import datetime
import os

def save_search_history(user_id, job_recommendations):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    history_data = []

    for _, row in job_recommendations.iterrows():
        history_data.append({
            "user_id": user_id,
            "job_id": row["job_link"],
            "interaction_score": 1,
            "timestamp": timestamp
        })

    history_df = pd.DataFrame(history_data)

    # Ghi dữ liệu vào file (append)
    history_df.to_csv("search_history.csv", mode="a", index=False, header=not os.path.exists("search_history.csv"))

# Gọi hàm lưu lịch sử
save_search_history(user_id, top_jobs)

print("Lịch sử tìm kiếm đã được lưu vào search_history.csv.")


Lịch sử tìm kiếm đã được lưu vào search_history.csv.
